# 2-2 自动微分机制
神经网络通常依赖反向传播求梯度来更新网络参数, 求梯度过程通常是一件复杂而容易出错的事情。

而深度学习框架可以帮助我们自动完成这种求梯度运算。

Pytorch一般通过反向传播backward方法, 实现这种求梯度计算。该方法求得的梯度将存在对应自变量张量的grad属性下。

除此之外, 也能调用torch.autograd.grad函数实现梯度计算。

## 一、利用backward方法求导数
backward方法通常在一个标量张量上调用, 该方法求得的梯度将存在对应自变量张量的grad属性下。

如果调用的张量非标量, 则要传入一个和它形状相同的gradient参数张量。

相当于用该gradient参数张量与调用张量作向量点乘法, 得到的标量结果再反向传播。因为如果结果也是向量, 那么向量对向量求导就会得到雅各比矩阵。

********************************************
这里需要说明一下的是, 只有张量同时满足叶张量和require_grad为True时才会在反向传播的时候自动填充grad, 如果想要对非叶张量求梯度, 那么就需要使用retain_grad()

### 1.标量的反向传播

In [1]:
import numpy as np
import torch

In [2]:
# 求 f(x) = a*x**2 + b*x + c的导数
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*x**2 + b*x + c

y.backward()
dy_dx = x.grad
print(dy_dx)
print(a.grad)

tensor(-2.)
None


## 2.非标量的反向传播

In [3]:
# x需要被求导
x = torch.tensor([[0.0, 0.0], [1.0, 2.0]], requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*x**2 + b*x + c

# 非标量反向传播的时候, 需要传入gradient
gradient = torch.tensor([[2, 2], [1.0, 1.0]])

y.backward(gradient=gradient) 
print(x.grad)

tensor([[-4., -4.],
        [ 0.,  2.]])


### 3.非标量的反向传播可以使用标量的反向传播实现

In [4]:
x = torch.tensor([[0.0, 0.0], [1.0, 2.0]], requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*x**2 + b*x + c

gradient = torch.tensor([[2, 2], [1.0, 1.0]])
z = torch.sum(y*gradient)
z.backward()  # 证明上一个传入gradient后, 将y变成标量使用了求和操作
print(x.grad)

tensor([[-4., -4.],
        [ 0.,  2.]])


## 二、利用autograd.grad方法求导数

In [5]:
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*x**2 + b*x + c

# create_graph设置为True, 将允许创建更高阶的导数
dy_dx = torch.autograd.grad(y, x, create_graph=True)[0]
print(type(dy_dx.data))
print(type(dy_dx))
print(dy_dx.detach())  # 后面的版本推荐使用detach(), 可以跟踪in-place操作

<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor(-2.)


In [6]:
x1 = torch.tensor(1.0,requires_grad = True) # x需要被求导
x2 = torch.tensor(2.0,requires_grad = True)

y1 = x1*x2
y2 = x1+x2

# 允许同时对多个自变量求导数
(dy1_dx1, dy1_dx2) = torch.autograd.grad(outputs=y1, inputs=[x1, x2], retain_graph=True)
print(dy1_dx1, dy1_dx2)

# 如果有多个因变量, 相当于把多个因变量的梯度结果求和
(dy12_dx1, dy1_dx2) = torch.autograd.grad(outputs=[y1, y2], inputs=[x1, x2])
print(dy12_dx1, dy1_dx2)

tensor(2.) tensor(1.)
tensor(3.) tensor(2.)


## 三、利用自动微分和优化器求最小值

In [7]:
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)

optimzer = torch.optim.SGD(params=[x], lr=0.01)

def f(x):
    result = a*x**2 + b*x + c
    return result

for i in range(500):
    optimzer.zero_grad()
    y = f(x)
    y.backward()
    optimzer.step()

print("y=", f(x).detach(), ";", "x=", x.detach())

y= tensor(0.) ; x= tensor(1.0000)


## 四、实验

In [8]:
# 测试optimizer.zero_grad作用和backward积累梯度机制
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)

def f(x):
    result = a*x**2 + b*x + c
    return result

y = f(x)
# 第一次backward
y.backward(retain_graph=True)
print("第一次x梯度:", x.grad)

# 第二次backward
y.backward()
print("第二次x梯度:", x.grad)

第一次x梯度: tensor(-2.)
第二次x梯度: tensor(-4.)


In [9]:
# 测试叶张量
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)
y = a*x**2 + b*x + c

y.backward()
print(a.requires_grad)
print(a.is_leaf)
print(a.grad)

False
True
None


In [10]:
# 测试梯度清除和detach
x = torch.tensor(0.0, requires_grad=True)
a = torch.tensor(1.0)
b = torch.tensor(-2.0)
c = torch.tensor(1.0)

optimzer = torch.optim.SGD(params=[x], lr=0.01)

def f(x):
    result = a*x**2 + b*x + c
    return result

for i in range(2):
    # 这里需要清除梯度
    if x.grad is not None:
        l = x.grad.detach_()
        x.grad.zero_()
        print("after detach:", x.grad)
    y = f(x)
    y.backward()
    print(x.grad)

# print("y=", f(x).detach(), ";", "x=", x.detach())

tensor(-2.)
after detach: tensor(0.)
tensor(-2.)


In [11]:
input = torch.ones([2, 2], requires_grad=False)
w1 = torch.tensor(2.0, requires_grad=True)
w2 = torch.tensor(3.0, requires_grad=True)
w3 = torch.tensor(4.0, requires_grad=True)

l1 = input * w1  # l1不是叶丈量, 但是是requires_grad, 因此其梯度计算完了就会被释放
l2 = l1 + w2
l3 = l1 * w3
l4 = l2 * l3
loss = l4.mean()


print(w1.data, w1.grad, w1.grad_fn)
# tensor(2.) None None

print(l1.data, l1.grad, l1.grad_fn)
# tensor([[2., 2.],
#         [2., 2.]]) None <MulBackward0 object at 0x000001EBE79E6AC8>

print(loss.data, loss.grad, loss.grad_fn)
# tensor(40.) None <MeanBackward0 object at 0x000001EBE79D8208>

tensor(2.) None None
tensor([[2., 2.],
        [2., 2.]]) None <MulBackward0 object at 0x7f156e8e8438>
tensor(40.) None <MeanBackward0 object at 0x7f156e8e8550>
